# Tutorial 13: Autoencoders — Learning Compressed Representations

In this notebook, we implement autoencoders from scratch and explore their variants:
- Basic (undercomplete) autoencoder
- Sparse autoencoder with KL penalty
- Denoising autoencoder
- Comparison with PCA
- Anomaly detection application

We use MNIST digits to visualize everything.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

np.random.seed(42)

# Style
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

## 1. Load and Prepare Data

In [ ]:
# Load MNIST (subset for speed)
from sklearn.datasets import load_digits
digits = load_digits()
X = digits.data / 16.0  # Normalize to [0, 1]
y = digits.target
input_dim = X.shape[1]  # 64 (8x8 images)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f'Training samples: {len(X_train)}')
print(f'Test samples: {len(X_test)}')
print(f'Input dimension: {input_dim}')

# Show some samples
fig, axes = plt.subplots(2, 10, figsize=(15, 3))
for i in range(10):
    idx = np.where(y_train == i)[0][0]
    axes[0, i].imshow(X_train[idx].reshape(8, 8), cmap='gray')
    axes[0, i].set_title(str(i))
    axes[0, i].axis('off')
    axes[1, i].bar(range(input_dim), X_train[idx], color='steelblue', width=1)
    axes[1, i].set_ylim(0, 1)
    axes[1, i].set_xticks([])
plt.suptitle('MNIST Digits (8x8) and Their Pixel Distributions', fontsize=14)
plt.tight_layout()
plt.show()

## 2. Activation Functions and Utilities

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-np.clip(x, -500, 500)))

def sigmoid_deriv(x):
    s = sigmoid(x)
    return s * (1 - s)

def relu(x):
    return np.maximum(0, x)

def relu_deriv(x):
    return (x > 0).astype(float)

# Xavier initialization
def xavier_init(fan_in, fan_out):
    limit = np.sqrt(6.0 / (fan_in + fan_out))
    return np.random.uniform(-limit, limit, (fan_out, fan_in))

## 3. Basic Autoencoder

Architecture: `input (64) → hidden (32) → latent (d) → hidden (32) → output (64)`

In [ ]:
class Autoencoder:
    """Simple autoencoder with one hidden layer in encoder and decoder."""
    
    def __init__(self, input_dim, hidden_dim, latent_dim):
        # Encoder weights
        self.W1 = xavier_init(input_dim, hidden_dim)
        self.b1 = np.zeros(hidden_dim)
        self.W2 = xavier_init(hidden_dim, latent_dim)
        self.b2 = np.zeros(latent_dim)
        
        # Decoder weights
        self.W3 = xavier_init(latent_dim, hidden_dim)
        self.b3 = np.zeros(hidden_dim)
        self.W4 = xavier_init(hidden_dim, input_dim)
        self.b4 = np.zeros(input_dim)
    
    def encode(self, x):
        """x -> z"""
        self.a1 = self.W1 @ x + self.b1
        self.h1 = relu(self.a1)
        self.a2 = self.W2 @ self.h1 + self.b2
        self.z = self.a2  # Linear latent layer
        return self.z
    
    def decode(self, z):
        """z -> x_hat"""
        self.a3 = self.W3 @ z + self.b3
        self.h2 = relu(self.a3)
        self.a4 = self.W4 @ self.h2 + self.b4
        self.x_hat = sigmoid(self.a4)  # Output in [0, 1]
        return self.x_hat
    
    def forward(self, x):
        z = self.encode(x)
        x_hat = self.decode(z)
        return x_hat, z
    
    def backward(self, x, x_hat, lr=0.001):
        """Compute gradients and update weights."""
        # Output gradient (MSE loss)
        d_out = 2 * (x_hat - x) / len(x)
        
        # Decoder output layer
        d_a4 = d_out * sigmoid_deriv(self.a4)
        dW4 = np.outer(d_a4, self.h2)
        db4 = d_a4
        d_h2 = self.W4.T @ d_a4
        
        # Decoder hidden
        d_a3 = d_h2 * relu_deriv(self.a3)
        dW3 = np.outer(d_a3, self.z)
        db3 = d_a3
        d_z = self.W3.T @ d_a3
        
        # Encoder latent (linear)
        d_a2 = d_z
        dW2 = np.outer(d_a2, self.h1)
        db2 = d_a2
        d_h1 = self.W2.T @ d_a2
        
        # Encoder hidden
        d_a1 = d_h1 * relu_deriv(self.a1)
        dW1 = np.outer(d_a1, x)
        db1 = d_a1
        
        # Update weights (gradient descent)
        self.W4 -= lr * dW4
        self.b4 -= lr * db4
        self.W3 -= lr * dW3
        self.b3 -= lr * db3
        self.W2 -= lr * dW2
        self.b2 -= lr * db2
        self.W1 -= lr * dW1
        self.b1 -= lr * db1

def train_autoencoder(ae, X_train, epochs=50, lr=0.01, verbose=True):
    """Train autoencoder with SGD."""
    losses = []
    for epoch in range(epochs):
        # Shuffle
        idx = np.random.permutation(len(X_train))
        epoch_loss = 0
        for i in idx:
            x = X_train[i]
            x_hat, z = ae.forward(x)
            loss = np.mean((x - x_hat)**2)
            ae.backward(x, x_hat, lr=lr)
            epoch_loss += loss
        epoch_loss /= len(X_train)
        losses.append(epoch_loss)
        if verbose and (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.6f}')
    return losses

In [ ]:
# Train with different latent dimensions
latent_dims = [2, 8, 16, 32]
autoencoders = {}
all_losses = {}

for d in latent_dims:
    print(f'\n--- Training Autoencoder with latent_dim={d} ---')
    ae = Autoencoder(input_dim=64, hidden_dim=32, latent_dim=d)
    losses = train_autoencoder(ae, X_train, epochs=30, lr=0.01)
    autoencoders[d] = ae
    all_losses[d] = losses

# Plot training curves
plt.figure(figsize=(10, 5))
for d, losses in all_losses.items():
    plt.plot(losses, label=f'd={d}')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.title('Training Loss vs Latent Dimension')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 4. Visualize Reconstructions

In [ ]:
def visualize_reconstructions(ae, X_test, n=10, title=''):
    """Show original vs reconstructed images."""
    fig, axes = plt.subplots(2, n, figsize=(15, 3))
    indices = np.random.choice(len(X_test), n, replace=False)
    
    for i, idx in enumerate(indices):
        x = X_test[idx]
        x_hat, _ = ae.forward(x)
        
        axes[0, i].imshow(x.reshape(8, 8), cmap='gray')
        axes[0, i].axis('off')
        if i == 0:
            axes[0, i].set_ylabel('Original', fontsize=12)
        
        axes[1, i].imshow(x_hat.reshape(8, 8), cmap='gray')
        axes[1, i].axis('off')
        if i == 0:
            axes[1, i].set_ylabel('Reconstructed', fontsize=12)
    
    plt.suptitle(title, fontsize=14)
    plt.tight_layout()
    plt.show()

for d in latent_dims:
    visualize_reconstructions(autoencoders[d], X_test, 
                             title=f'Autoencoder Reconstructions (d={d})')

## 5. Latent Space Visualization (d=2)

In [ ]:
# Encode all test data with the d=2 autoencoder
ae_2d = autoencoders[2]
Z_test = np.array([ae_2d.encode(x) for x in X_test])

# Compare with PCA
pca_2d = PCA(n_components=2)
Z_pca = pca_2d.fit_transform(X_test)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))

# Autoencoder latent space
scatter1 = ax1.scatter(Z_test[:, 0], Z_test[:, 1], c=y_test, 
                       cmap='tab10', alpha=0.7, s=20)
ax1.set_title('Autoencoder Latent Space (d=2)', fontsize=14)
ax1.set_xlabel('z₁')
ax1.set_ylabel('z₂')
plt.colorbar(scatter1, ax=ax1, label='Digit')

# PCA projection
scatter2 = ax2.scatter(Z_pca[:, 0], Z_pca[:, 1], c=y_test, 
                       cmap='tab10', alpha=0.7, s=20)
ax2.set_title('PCA Projection (d=2)', fontsize=14)
ax2.set_xlabel('PC₁')
ax2.set_ylabel('PC₂')
plt.colorbar(scatter2, ax=ax2, label='Digit')

plt.suptitle('Nonlinear AE vs Linear PCA: 2D Representations', fontsize=16)
plt.tight_layout()
plt.show()

print(f'PCA explained variance ratio: {pca_2d.explained_variance_ratio_.sum():.3f}')

## 6. Reconstruction Quality: Autoencoder vs PCA

In [ ]:
# Compare reconstruction MSE
dims = [2, 4, 8, 16, 32]
mse_ae = []
mse_pca = []

for d in dims:
    # PCA
    pca = PCA(n_components=d)
    Z_pca = pca.fit_transform(X_train)
    X_recon_pca = pca.inverse_transform(pca.transform(X_test))
    mse_pca.append(np.mean((X_test - X_recon_pca)**2))
    
    # Autoencoder (train if not already)
    if d not in autoencoders:
        ae = Autoencoder(64, 32, d)
        train_autoencoder(ae, X_train, epochs=30, lr=0.01, verbose=False)
        autoencoders[d] = ae
    
    ae = autoencoders[d]
    errors = [np.mean((x - ae.forward(x)[0])**2) for x in X_test]
    mse_ae.append(np.mean(errors))

plt.figure(figsize=(10, 6))
plt.plot(dims, mse_pca, 'bo-', label='PCA', linewidth=2, markersize=8)
plt.plot(dims, mse_ae, 'rs-', label='Autoencoder', linewidth=2, markersize=8)
plt.xlabel('Latent Dimension', fontsize=13)
plt.ylabel('Test MSE', fontsize=13)
plt.title('Reconstruction Quality: Autoencoder vs PCA', fontsize=14)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.xticks(dims)
plt.show()

for d, pca_err, ae_err in zip(dims, mse_pca, mse_ae):
    improvement = (pca_err - ae_err) / pca_err * 100
    print(f'd={d:2d}: PCA MSE={pca_err:.5f}, AE MSE={ae_err:.5f}, AE improvement={improvement:+.1f}%')

## 7. Denoising Autoencoder

Train the autoencoder to reconstruct clean images from corrupted ones.

In [ ]:
def add_masking_noise(x, noise_level=0.3):
    """Zero out random pixels."""
    mask = np.random.binomial(1, 1 - noise_level, size=x.shape)
    return x * mask

def add_gaussian_noise(x, sigma=0.2):
    """Add Gaussian noise."""
    return np.clip(x + np.random.normal(0, sigma, size=x.shape), 0, 1)

# Show noise types
fig, axes = plt.subplots(3, 8, figsize=(14, 5))
for i in range(8):
    x = X_test[i]
    axes[0, i].imshow(x.reshape(8, 8), cmap='gray')
    axes[0, i].axis('off')
    
    x_mask = add_masking_noise(x, 0.4)
    axes[1, i].imshow(x_mask.reshape(8, 8), cmap='gray')
    axes[1, i].axis('off')
    
    x_gauss = add_gaussian_noise(x, 0.3)
    axes[2, i].imshow(x_gauss.reshape(8, 8), cmap='gray')
    axes[2, i].axis('off')

axes[0, 0].set_ylabel('Clean', fontsize=12)
axes[1, 0].set_ylabel('Masking', fontsize=12)
axes[2, 0].set_ylabel('Gaussian', fontsize=12)
plt.suptitle('Noise Types for Denoising Autoencoder', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Train denoising autoencoder
dae = Autoencoder(input_dim=64, hidden_dim=32, latent_dim=16)

dae_losses = []
for epoch in range(50):
    idx = np.random.permutation(len(X_train))
    epoch_loss = 0
    for i in idx:
        x_clean = X_train[i]
        x_noisy = add_masking_noise(x_clean, noise_level=0.3)
        
        # Forward with noisy input
        x_hat, z = dae.forward(x_noisy)
        
        # Loss against CLEAN input
        loss = np.mean((x_clean - x_hat)**2)
        
        # Backward (target is clean input)
        dae.backward(x_clean, x_hat, lr=0.01)
        epoch_loss += loss
    
    epoch_loss /= len(X_train)
    dae_losses.append(epoch_loss)
    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch+1}/50, Loss: {epoch_loss:.6f}')

# Visualize denoising
fig, axes = plt.subplots(3, 8, figsize=(14, 5))
for i in range(8):
    x = X_test[i]
    x_noisy = add_masking_noise(x, 0.3)
    x_hat, _ = dae.forward(x_noisy)
    
    axes[0, i].imshow(x.reshape(8, 8), cmap='gray')
    axes[0, i].axis('off')
    axes[1, i].imshow(x_noisy.reshape(8, 8), cmap='gray')
    axes[1, i].axis('off')
    axes[2, i].imshow(x_hat.reshape(8, 8), cmap='gray')
    axes[2, i].axis('off')

axes[0, 0].set_ylabel('Clean', fontsize=12)
axes[1, 0].set_ylabel('Corrupted', fontsize=12)
axes[2, 0].set_ylabel('Denoised', fontsize=12)
plt.suptitle('Denoising Autoencoder Results', fontsize=14)
plt.tight_layout()
plt.show()

## 8. Anomaly Detection with Autoencoders

Train on one class of digits, detect other classes as anomalies.

In [ ]:
# Train autoencoder on digit '1' only
normal_digit = 1
X_normal = X_train[y_train == normal_digit]
X_test_normal = X_test[y_test == normal_digit]
X_test_anomaly = X_test[y_test != normal_digit]

print(f'Training on digit {normal_digit}: {len(X_normal)} samples')
print(f'Test normal: {len(X_test_normal)}, Test anomaly: {len(X_test_anomaly)}')

# Train
ae_anomaly = Autoencoder(64, 32, 8)
losses = train_autoencoder(ae_anomaly, X_normal, epochs=50, lr=0.01)

# Compute reconstruction errors
errors_normal = [np.mean((x - ae_anomaly.forward(x)[0])**2) for x in X_test_normal]
errors_anomaly = [np.mean((x - ae_anomaly.forward(x)[0])**2) for x in X_test_anomaly]

# Plot distributions
plt.figure(figsize=(10, 5))
plt.hist(errors_normal, bins=30, alpha=0.6, label=f'Normal (digit {normal_digit})', color='green', density=True)
plt.hist(errors_anomaly, bins=30, alpha=0.6, label='Anomaly (other digits)', color='red', density=True)
threshold = np.percentile(errors_normal, 95)
plt.axvline(threshold, color='black', linestyle='--', linewidth=2, label=f'Threshold (95th percentile)')
plt.xlabel('Reconstruction Error (MSE)', fontsize=13)
plt.ylabel('Density', fontsize=13)
plt.title('Anomaly Detection: Reconstruction Error Distribution', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

# Detection metrics
tp = np.sum(np.array(errors_anomaly) > threshold)
fn = np.sum(np.array(errors_anomaly) <= threshold)
fp = np.sum(np.array(errors_normal) > threshold)
tn = np.sum(np.array(errors_normal) <= threshold)

precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
print(f'\nPrecision: {precision:.3f}')
print(f'Recall: {recall:.3f}')
print(f'F1: {2*precision*recall/(precision+recall):.3f}' if (precision+recall) > 0 else 'F1: 0')

## 9. Information Bottleneck Visualization

How does the reconstruction error change with latent dimension?

In [ ]:
# Rate-distortion curve approximation
dims_to_test = [1, 2, 4, 8, 12, 16, 24, 32, 48, 64]
distortions = []

for d in dims_to_test:
    ae = Autoencoder(64, max(d+4, 32), d)
    train_autoencoder(ae, X_train, epochs=30, lr=0.01, verbose=False)
    errors = [np.mean((x - ae.forward(x)[0])**2) for x in X_test[:200]]
    distortions.append(np.mean(errors))
    print(f'd={d:2d}: MSE={distortions[-1]:.5f}')

plt.figure(figsize=(10, 6))
plt.plot(dims_to_test, distortions, 'bo-', linewidth=2, markersize=8)
plt.xlabel('Latent Dimension (Rate proxy)', fontsize=13)
plt.ylabel('Reconstruction MSE (Distortion)', fontsize=13)
plt.title('Rate-Distortion Trade-off in Autoencoders', fontsize=14)
plt.grid(True, alpha=0.3)

# Mark input dimension
plt.axvline(x=64, color='red', linestyle='--', alpha=0.5, label='Input dim (64)')
plt.legend(fontsize=12)
plt.show()

## 10. Latent Space Interpolation

Interpolate between two digits in latent space to see smooth transitions.

In [ ]:
ae = autoencoders[16]  # Use d=16 autoencoder

# Pick two different digits
idx_3 = np.where(y_test == 3)[0][0]
idx_8 = np.where(y_test == 8)[0][0]

x_start = X_test[idx_3]
x_end = X_test[idx_8]

z_start = ae.encode(x_start)
z_end = ae.encode(x_end)

# Interpolate
n_steps = 10
fig, axes = plt.subplots(1, n_steps, figsize=(15, 2))

for i, alpha in enumerate(np.linspace(0, 1, n_steps)):
    z_interp = (1 - alpha) * z_start + alpha * z_end
    x_interp = ae.decode(z_interp)
    axes[i].imshow(x_interp.reshape(8, 8), cmap='gray')
    axes[i].set_title(f'{alpha:.1f}', fontsize=10)
    axes[i].axis('off')

plt.suptitle('Latent Space Interpolation: 3 → 8', fontsize=14)
plt.tight_layout()
plt.show()

print('Note: Standard autoencoders may have discontinuous latent spaces.')
print('VAEs address this by regularizing the latent space with KL divergence.')

## Key Insights

1. **Bottleneck matters**: Smaller latent dimension = more compression = higher reconstruction error
2. **Nonlinear > Linear**: Autoencoders outperform PCA especially at low dimensions
3. **Denoising helps**: Training with corruption forces robust feature learning
4. **Anomaly detection**: High reconstruction error indicates out-of-distribution inputs
5. **Latent space structure**: Standard AEs have irregular latent spaces; VAEs fix this

**Next**: Variational Autoencoders add a probabilistic framework, enabling generation and smooth latent spaces.